In [1]:
import torch
from torchvision import transforms

from aim_perception.training import Trainer
from aim_perception.evaluation import Evaluation
from aim_perception.models import ResNet34, ModelWrapper, LinearHead
from aim_perception.loading import AimDataset, AimDatasetConstructor


In [2]:
abs_path = '/Users/mjfagundo/Documents/vehicle_dataset'
image_path = 'data'
label_path = 'ground_truth.csv'

In [3]:

dataset_constructor = AimDatasetConstructor(
    root_dir=abs_path,
    csv_path=label_path,
    data_subdir=image_path,
    transforms=[
        transforms.ToTensor(),
        transforms.Resize(size=(96, 96)),
        transforms.Normalize(mean=[0.4886, 0.4855, 0.4838], std=[0.2456, 0.2443, 0.2490])
    ]
)

Train percent: 75.00150253026095
Val percent: 14.99885807700167
Test percent: 9.99963939273737


In [4]:
train, val, test = dataset_constructor.get_all_datasets()

In [5]:
from aim_perception.models import ModelWrapper, LinearHead
# from torchvision.models import resnet50, ResNet50_Weights

epochs = 6
batch_size = 64
learning_rate = 1e-1
weight_decay = 1e-4


# Create loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size*2, shuffle=True, num_workers=2)

# Create model
# model = ModelWrapper(resnet50(weights=ResNet50_Weights.IMAGENET1K_V2), LinearHead(input_size=1000, num_targets=10))
model = ModelWrapper(ResNet34(in_channels=3, depthwise_separable=True), LinearHead(input_size=512, num_targets=10))

# Optimizer and loss
criterion = torch.nn.CrossEntropyLoss(weight=train.get_class_weights())
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)



In [6]:
trainer = Trainer(epochs=6, validate_every=50, criterion=criterion, optimizer=optimizer, scheduler=scheduler, wandb_project='test')
trainer(model=model, train_loader=train_loader, val_loader=val_loader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fagundo. Use `wandb login --relogin` to force relogin


[1,    51] train loss: 3.401 | val loss: 2.344 | val bal-acc: 0.100
